In [1]:
# Importing libraries
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
 
# Load the dataset
dataset = pd.read_csv('kurla_cleaned.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
dataset=dataset.drop(dataset[['Date']],axis=1)
# Prepare the data
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
 
# Split the dataset into train and test sets
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
 
# Convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)
 
# Reshape the data into X=t and Y=t+1
look_back = 24
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
 
# Reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))
 
# Create the LSTM model
model = Sequential()
model.add(LSTM(4, input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
 
# Train the model
model.fit(trainX, trainY, epochs=10, batch_size=1, verbose=2)
 
# Make predictions on test data
testPredict = model.predict(testX)
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
 
# Calculate RMSE on test data
rmse = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test RMSE: %.2f' % rmse)

# Calculate the accuracy on test data
threshold = 20
testPredict_high = (testPredict > threshold).astype(int)
testY_high = (testY > threshold).astype(int)
accuracy = (testPredict_high == testY_high).mean() * 100
print('Test Accuracy: %.2f%%' % accuracy)
 
# Plot the actual and predicted values
plt.plot(testY[0])
plt.plot(testPredict)
plt.legend(['Actual', 'Predicted'])
plt.show()

Epoch 1/10
14026/14026 - 76s - loss: 0.0045 - 76s/epoch - 5ms/step
Epoch 2/10
14026/14026 - 74s - loss: 0.0035 - 74s/epoch - 5ms/step
Epoch 3/10
14026/14026 - 76s - loss: 0.0035 - 76s/epoch - 5ms/step
Epoch 4/10
14026/14026 - 73s - loss: 0.0035 - 73s/epoch - 5ms/step
Epoch 5/10
14026/14026 - 72s - loss: 0.0034 - 72s/epoch - 5ms/step
Epoch 6/10
14026/14026 - 75s - loss: 0.0034 - 75s/epoch - 5ms/step
Epoch 7/10
14026/14026 - 74s - loss: 0.0034 - 74s/epoch - 5ms/step
Epoch 8/10
14026/14026 - 74s - loss: 0.0034 - 74s/epoch - 5ms/step
Epoch 9/10
14026/14026 - 75s - loss: 0.0033 - 75s/epoch - 5ms/step
Epoch 10/10
14026/14026 - 74s - loss: 0.0034 - 74s/epoch - 5ms/step
109/109 [==============================] - 6s 3ms/step


ValueError: non-broadcastable output operand with shape (3488,1) doesn't match the broadcast shape (3488,11)